# Exploring the Entity Extraction System

In [1]:
import pandas as pd
import numpy as np
import pickle
import re
import nltk
import difflib
from neo4j.v1 import GraphDatabase, basic_auth

# Neo4j driver and client
driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth("neo4j", "naturalmed"))
session = driver.session()

In [22]:
ENTITY_LIST = [
    'Medicine',
    'Disease',
    'Food',
    'Context',
    'HerbSuplement',
    'LaboratoryTest',
    'Pharmacokinetics',
    'ScientificName',
    'Synonymous'
]

QUESTIONS = [
    'What is ([A-Z].*)\?',
    'What is the relation between ([A-Z].*) and ([A-Z].*)\?'
]

In [3]:
entities = []
types = []

for entity in ENTITY_LIST:
    prop = 'name' if entity == 'Medicine' else 'id' 
    res = session.run("MATCH (n:%s) RETURN n.%s;" % (entity, prop))
    
    for e in res:
        entities.append(e.values()[0])
        types.append(entity)

dframe = pd.DataFrame({
    'entity': pd.Series(entities),
    'type': pd.Series(types)
})

with open("../Dumps/entities.pickle", "wb") as fp:
    pickle.dump(dframe, fp)

In [4]:
dframe.head()

,entity,type
0,Abscess Root,Medicine
1,Abuta,Medicine
2,Acacia,Medicine
3,Acacia Rigidula,Medicine
4,Acai,Medicine


In [5]:
dframe.count()

entity    24076
type      24076
dtype: int64

In [6]:
def comp_str(arr, string):
    """ Compare the matching from 0 to 1 between two strings
    """
    sm = difflib.SequenceMatcher(None)
    
    for e in arr:
        sm.set_seq1(str(e))
        sm.set_seq2(string)
        
        yield sm.ratio()

In [7]:
def compile_question(string):
    """ Compiles a question and returns a list of the entities matched on it.
    """
    for question in QUESTIONS:
        matching = re.match(question, string) 
        if matching:
            return list(matching.groups())
    return None

In [20]:
compile_question("What is Marijuana?")

['Marijuana']

In [23]:
compile_question("What is the relation between Marijuana and Depression?")

['Marijuana', 'Depression']

In [8]:
def score_entity(dframe, entity):
    """ Scores the entity in relation to the entities table
    """
    dframe['confidence'] = list(comp_str(dframe['entity'], entity.title()))
    return dframe.sort_values(by='confidence', ascending=False).head(n=5)

In [44]:
def question(question):
    entities = compile_question(question)
    for entity in entities:
        print("Entity: {}\n".format(entity))
        print(score_entity(dframe, entity))
        print("\n")

## Examples

In [45]:
question("What is Açai?")

Entity: Açai

       entity        type  confidence
4        Acai    Medicine    0.750000
6230     Açaï  Synonymous    0.750000
6238    Assai  Synonymous    0.666667
13287   Alasi  Synonymous    0.666667
10375  Arandi  Synonymous    0.600000




In [46]:
question("What is Blue Bells?")

Entity: Blue Bells

            entity        type  confidence
6188    Blue Bells  Synonymous    1.000000
18609    Blue Balm  Synonymous    0.736842
20833   Blue Curls  Synonymous    0.700000
14896     Hen Bell  Synonymous    0.666667
12842  Blue Mallee  Synonymous    0.666667




In [47]:
question("What is Marijuana?")

Entity: Marijuana

          entity           type  confidence
743    Marijuana       Medicine    1.000000
2840   Marijuana  HerbSuplement    1.000000
17494  Mariguana     Synonymous    0.888889
17495  Marihuana     Synonymous    0.888889
7330     Maranta     Synonymous    0.750000




In [48]:
question("What is the relation between Marijuana and Depression?")

Entity: Marijuana

          entity           type  confidence
743    Marijuana       Medicine    1.000000
2840   Marijuana  HerbSuplement    1.000000
17494  Mariguana     Synonymous    0.888889
17495  Marihuana     Synonymous    0.888889
7330     Maranta     Synonymous    0.750000


Entity: Depression

                    entity        type  confidence
1232            Depression     Disease    1.000000
1964     Mental Depression     Disease    0.740741
23251              Cresson  Synonymous    0.705882
1763   Atypical Depression     Disease    0.689655
11344             Espresso  Synonymous    0.666667




In [137]:
#def ngram_it(text, n=3):
#    """ Returns a list of ngrams of sizes 1 to n
#    """
#    ngrams = []
#    tokens = nltk.word_tokenize(text)
#    
#    for i in range(1, n + 1):
#        ngrams += list(map(" ".join, nltk.ngrams(tokens, i)))
#    
#    return ngrams

In [ ]:
# Close the Neo4j Session
#session.close()